<a href="https://colab.research.google.com/github/YapingWu/GoogleColab/blob/main/lstm/exp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考资料：[通过keras例子理解LSTM 循环神经网络(RNN)](https://blog.csdn.net/zwqjoy/article/details/80493341)

学习字母表的一个简单的序列预测问题。也就是说，根据字母表的字母，可以预测字母表的下一个字母。

In [ ]:
# 获取模块的版本号
import platform 
import keras
print(platform.python_version()) # 3.6.9
print(keras.__version__) # 2.4.3

3.6.9
2.4.3


导入需要使用的所用类和函数

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

对随机数生成器选定随机数种子，以确保每次执行代码时结果都是相同的。

In [ ]:
# fix random seed for reproducibility
numpy.random.seed(7)

## 1 定义数据集——字母表

In [ ]:
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
# 神经网络是对数字建模，因此我们需要将字母表中的字母映射到整数值（把字母映射为数字）
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
# 创建一个反向查找，以便将预测转换回字符
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

## 2 数据预处理——创建输入输出



In [ ]:
# prepare the dataset of input to output pairs encoded as integers
# 定义输入序列长度
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in]) # 为什么？
    dataY.append(char_to_int[seq_out])
    print(seq_in, '->', seq_out)

### 2.1 转换输入序列的格式






将NumPy数组重新构造为LSTM网络所期望的格式，即[samples示例, time steps时间步数, features特征]。

In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))

### 2.2 归一化

整数值归一化到0～1的区间上，这是LSTM网络使用的s形激活函数（sigmoid）的范围。

In [ ]:
# normalize
X = X / float(len(alphabet))

### 2.3 对输出进行编码

可以把这个问题看作是一个序列分类任务，其中26个字母代表一个不同的类。因此，我们用keras的内置的to_categorical()函数把输出output(y)进行one－hot编码(one-hot指n维单位向量a=(0,…,0,1,0,…,0))作为输出层的结果。

In [ ]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## 3 单字符——单字符映射的简单LSTM

设计一个简单的LSTM，学习如何根据一个字符的上下文来预测字母表中的下一个字符。

问题定义：一些单字母的随机集合作为输入，另一些单字母作为输出，由输入输出对组成。正如我们所看到的，这对于LSTM来说是一个很难用来学习的结构。

定义一个LSTM网络。  
单元数：32个(the LSTM units are the “memory units” or you can just call them the neurons.)  
输出层：激活函数：softmax  
损失函数：使用Keras中的对数损失函数(称为“分类交叉熵”(categorical_crossentropy))  
优化方法：Adam

该模型以500批次(epochs)，每批次数据输入大小(batch)为1的形式训练

In [ ]:
# create and fit the model
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, batch_size=1, verbose=2)

# verbose：日志显示
# verbose = 0 为不在标准输出流输出日志信息
# verbose = 1 为输出进度条记录
# verbose = 2 为每个epoch输出一行记录
# 注意： 默认为 1

在我们训练模型之后，我们可以对整个训练集的性能进行评估和总结。

In [ ]:
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 80.00%


然后，我们可以通过网络重新运行训练数据，并生成预测，将输入和输出对转换回原来的字符格式，以获得关于网络如何了解问题的视觉效果。

In [ ]:
# demonstrate some model predictions
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction) # 分类结果中概率最大的类别
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

我们可以看到，这个问题对于网络来说确实是很困难的。
原因是可怜的lstm单元根本没有可以利用的上下文章信息。
每个输入输出模式都以随机的顺序显示在网络中，并且网络的状态在每个模式之后被重置(每个批处理的每个批次包含一个模式)。

这是对LSTM网络架构的滥用，因为我们把它当作了一个标准的多层感知器。

接下来，让我们尝试一个不同的问题框架，以便为网络提供更多的序列来学习。

## 4 三字符特征——单字符的映射的简单LSTM

在多层感知器中添加更多上下文最流行的方法是特征窗口方法(Feature Window method)。

即序列中的前面步骤的输出被作为附加的输入特性提供给网络。我们可以用相同的技巧，为LSTM网络提供更多的上下文。

在这里，我们将序列长度从1增加到3，例如:
我们把输入从一个字符升到三个字符。

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 3

然后将序列中的每个元素作为网络的一个新输入特性提供给它。这需要修改输入序列在数据准备步骤中的reshape:

In [ ]:
# Naive LSTM to learn three-char window to one-char mapping
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
# fix random seed for reproducibility
numpy.random.seed(7)
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# prepare the dataset of input to output pairs encoded as integers
seq_length = 3
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    # print(seq_in, "->", seq_out)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), 1, seq_length))
# print("dataX:", dataX, sep='\n')
# print("X:", X, sep='\n')
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# create and fit the model
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, batch_size=1, verbose=0)
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))
# demonstrate some model predictions
for pattern in dataX:
    x = numpy.reshape(pattern, (1, 1, len(pattern)))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

Model Accuracy: 82.61%
['A', 'B', 'C'] -> D
['B', 'C', 'D'] -> E
['C', 'D', 'E'] -> F
['D', 'E', 'F'] -> G
['E', 'F', 'G'] -> H
['F', 'G', 'H'] -> I
['G', 'H', 'I'] -> J
['H', 'I', 'J'] -> K
['I', 'J', 'K'] -> L
['J', 'K', 'L'] -> M
['K', 'L', 'M'] -> N
['L', 'M', 'N'] -> O
['M', 'N', 'O'] -> P
['N', 'O', 'P'] -> Q
['O', 'P', 'Q'] -> Q
['P', 'Q', 'R'] -> S
['Q', 'R', 'S'] -> T
['R', 'S', 'T'] -> U
['S', 'T', 'U'] -> V
['T', 'U', 'V'] -> X
['U', 'V', 'W'] -> Z
['V', 'W', 'X'] -> Z
['W', 'X', 'Y'] -> Z


## 5 keras实践循环的正确打开方式！

在keras中，利用lstm的关键是以时间序列(time steps)的方法来提供上下文，而不是像其他网络结构(CNN)一样，通过windowed features的方式。

我们这次唯一改变的地方是下面这里：

In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))

time steps这个参数，我们设置了3，而不是前面的1。

不同之处是，对输入数据的reshape是将输入序列作为一个特性的time step序列，而不是多个特性的单一time step。也就是说我们把ABC 看成独立的一个特征组成的多个时间序列，而不是把ABC看成一个多个特征组成一个时间序列。

这就是keras中LSTM循环神经网络的正确打开的方式。我的理解是，这样在训练 ABC——D的时候，BCD，CDE，都可以发挥作用。而最开始那种使用方法，只是利用了ABC——D这样一个训练样本。

In [ ]:
# Naive LSTM to learn three-char time steps to one-char mapping
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
# fix random seed for reproducibility
numpy.random.seed(7)
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# prepare the dataset of input to output pairs encoded as integers
seq_length = 3
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    # print(seq_in, '->', seq_out)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))
# print("X:", X, sep='\n')
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# create and fit the model
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, batch_size=1, verbose=0)
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))
# demonstrate some model predictions
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

Model Accuracy: 100.00%
['A', 'B', 'C'] -> D
['B', 'C', 'D'] -> E
['C', 'D', 'E'] -> F
['D', 'E', 'F'] -> G
['E', 'F', 'G'] -> H
['F', 'G', 'H'] -> I
['G', 'H', 'I'] -> J
['H', 'I', 'J'] -> K
['I', 'J', 'K'] -> L
['J', 'K', 'L'] -> M
['K', 'L', 'M'] -> N
['L', 'M', 'N'] -> O
['M', 'N', 'O'] -> P
['N', 'O', 'P'] -> Q
['O', 'P', 'Q'] -> R
['P', 'Q', 'R'] -> S
['Q', 'R', 'S'] -> T
['R', 'S', 'T'] -> U
['S', 'T', 'U'] -> V
['T', 'U', 'V'] -> W
['U', 'V', 'W'] -> X
['V', 'W', 'X'] -> Y
['W', 'X', 'Y'] -> Z


它已经学会了用字母表中的三个字母来预测下一个字母的顺序。它可以显示字母表中的任意三个字母的随机序列，并预测下一个字母。

我们还没有展示出循环神经网络的强大之处，因为上面这个问题我们用多层感知器，足够多的神经元，足够多的迭代次数也可以很好的解决。（三层神经网络拟合任意可以表示的函数）

LSTM网络是有状态的。它们应该能够学习整个字母表序列，但是在默认情况下，keras在每次训练之后重新设置网络状态。

那么接下来就是展示循环神经网络的独到之处！！

## 6 一个批处理中的LSTM状态

keras实现的LSTM在每一个batch以后，都重置了LSTM的状态。

这表明，如果我们的批处理大小足够容纳所有输入模式，如果所有输入模式都按顺序排序，LSTM就可以使用序列中的序列上下文来更好地学习序列。

通过修改第一个示例来学习一对一映射，并将批处理大小从1增加到训练数据集的大小，我们可以很容易地演示这一点。

此外，在每个epoch前，keras都重置了训练数据集。为了确保训练数据模式保持顺序，我们可以禁用这种洗牌。

In [ ]:
model.fit(X, y, epochs=5000, batch_size=len(dataX), verbose=2, shuffle=False)

该网络将使用within-batch批序列来学习字符的映射，但在进行预测时，这个上下文将无法用于网络。我们可以对网络进行评估，以确定网络在随机序列和顺序序列的预测能力。

In [ ]:
# Naive LSTM to learn one-char to one-char mapping with all data in each batch
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
# fix random seed for reproducibility
numpy.random.seed(7)
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# prepare the dataset of input to output pairs encoded as integers
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    # print(seq_in, '->', seq_out)
# convert list of lists to array and pad sequences if needed
X = pad_sequences(dataX, maxlen=seq_length, dtype='float32')
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (X.shape[0], seq_length, 1))
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# create and fit the model
model = Sequential()
model.add(LSTM(16, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=5000, batch_size=len(dataX), verbose=0, shuffle=False)
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))
# demonstrate some model predictions
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)
# demonstrate predicting random patterns
print("Test a Random Pattern:")
for i in range(0,20):
    pattern_index = numpy.random.randint(len(dataX))
    pattern = dataX[pattern_index]
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

Model Accuracy: 100.00%
['A'] -> B
['B'] -> C
['C'] -> D
['D'] -> E
['E'] -> F
['F'] -> G
['G'] -> H
['H'] -> I
['I'] -> J
['J'] -> K
['K'] -> L
['L'] -> M
['M'] -> N
['N'] -> O
['O'] -> P
['P'] -> Q
['Q'] -> R
['R'] -> S
['S'] -> T
['T'] -> U
['U'] -> V
['V'] -> W
['W'] -> X
['X'] -> Y
['Y'] -> Z
Test a Random Pattern:
['P'] -> Q
['E'] -> F
['W'] -> X
['D'] -> E
['T'] -> U
['X'] -> Y
['H'] -> I
['O'] -> P
['X'] -> Y
['I'] -> J
['O'] -> P
['K'] -> L
['I'] -> J
['H'] -> I
['G'] -> H
['E'] -> F
['Q'] -> R
['H'] -> I
['M'] -> N
['A'] -> B
